In [1]:



import pandas as pd
import numpy as np
import jsonlines
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch_optimizer as optim


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from importlib import reload
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', '{:0.3f}'.format)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.width = 0
import warnings
warnings.filterwarnings('ignore')
from facebook_hateful_memes_detector.utils.globals import set_global, get_global
set_global("cache_dir", "/home/ahemf/cache/cache")

import facebook_hateful_memes_detector
reload(facebook_hateful_memes_detector)

from facebook_hateful_memes_detector.utils import read_json_lines_into_df, in_notebook, set_device
from facebook_hateful_memes_detector.models import Fasttext1DCNNModel, MultiImageMultiTextAttentionEarlyFusionModel
from facebook_hateful_memes_detector.preprocessing import TextImageDataset, my_collate, get_datasets, get_image2torchvision_transforms, TextAugment
from facebook_hateful_memes_detector.training import *
from facebook_hateful_memes_detector.preprocessing import DefinedRotation, QuadrantCut



https://discuss.pytorch.org/t/how-to-implement-torch-optim-lr-scheduler-cosineannealinglr/28797/11

# Params

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_device(device)

choice_probas = {"keyboard": 0.1, "char_substitute": 0.0, "char_insert": 0.1, "char_swap": 0.1, "ocr": 0.0, "char_delete": 0.1,
                 "fasttext": 0.0, "glove_twitter": 0.0, "glove_wiki": 0.0, "word2vec": 0.0, "split": 0.1,
                 "stopword_insert": 0.3, "word_join": 0.1, "word_cutout": 0.8,
                 "text_rotate": 0.5, "sentence_shuffle": 0.5, "one_third_cut": 0.3, "half_cut":0.1}
preprocess_text = TextAugment([0.05, 0.0, 0.1, 0.3, 0.35, 0.2], choice_probas, fasttext_file="wiki-news-300d-1M-subword.bin")

im_transform = transforms.RandomChoice([transforms.Grayscale(num_output_channels=3), 
                                        transforms.RandomHorizontalFlip(p=1.0),
                                        lambda x: x,
                                        transforms.Compose([transforms.Resize(480), transforms.RandomCrop(400)]), 
                                        DefinedRotation(15), QuadrantCut()])

data = get_datasets(data_dir="../data/", train_text_transform=preprocess_text, train_image_transform=im_transform, 
                    test_text_transform=None, test_image_transform=None, 
                    cache_images = True, use_images = True, dev=False, 
                    keep_original_text=False, keep_original_image=False, 
                    keep_processed_image=True, keep_torchvision_image=True,)



In [3]:
sgd = torch.optim.SGD
sgd_params = dict(lr=2e-2, momentum=0.9, dampening=0, weight_decay=0, nesterov=False)

rangerQH = optim.RangerQH
rangerQHparams = dict(lr=1e-3, betas=(0.9, 0.999), nus=(.7, 1.0),
    weight_decay=0.0,
    k=6,
    alpha=.5,
    decouple_weight_decay=True,
    eps=1e-8,)

adam = torch.optim.Adam
adam_params = params=dict(lr=5e-4, weight_decay=1e-5)

adamw = torch.optim.AdamW
adamw_params = dict(lr=1e-4, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-2)

novograd = optim.NovoGrad
novograd_params = dict(lr= 1e-3,
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0,
    grad_averaging=False,
    amsgrad=False,)

qhadam = optim.QHAdam
qhadam_params = dict(lr= 1e-3,
    betas=(0.9, 0.999),
    nus=(1.0, 1.0),
    weight_decay=0,
    decouple_weight_decay=False,
    eps=1e-8,)

radam = optim.RAdam
radam_params = dict(lr= 1e-3,
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0,)

yogi = optim.Yogi
yogi_params = dict(lr= 1e-2,
    betas=(0.9, 0.999),
    eps=1e-3,
    initial_accumulator=1e-6,
    weight_decay=0)




In [10]:
batch_size=256
epochs = 20
optimizer = adam
optimizer_params = adam_params

scheduler_init_fn = get_multistep_lr([5, 7, 10, 17], gamma=0.1) # get_cosine_schedule_with_warmup # get_cosine_with_hard_restarts_schedule_with_warmup
scheduler_init_fn = get_cosine_schedule_with_warmup()
augmentation_weights = {"None": 1.0}

lr_strategy = {"im_models": {"torchvision_resnet18": {"7": {"lr": optimizer_params["lr"]/750},
                                                      "8": {"lr": optimizer_params["lr"]/250},
                                                      "lr": optimizer_params["lr"]/2000},
                            "lr": optimizer_params["lr"]/2000},
              }

# {"lr": optimizer_params["lr"]/500}



# ImageAttentionModel

In [11]:
model_fn = model_builder(MultiImageMultiTextAttentionEarlyFusionModel, 
                         dict(image_models=[{"model": "torchvision_resnet18", "large_rf": True, "finetune": True}], num_classes=2, 
                              text_models=[dict(cls=Fasttext1DCNNModel, params=dict(classifier_dims=256, 
                                                                                    num_classes=2, n_tokens_out=16,
                                                                                    n_layers=2, final_layer_builder=lambda *args: None,
                                                                                    gaussian_noise=0.2, dropout=0.15, 
                                                                                    embedding_dims=256, internal_dims=256, featurizer="cnn",),
                                               in_channels=256, in_tokens=16)],
                              internal_dims=256, classifier_dims=256,
                              n_tokens_out=16, n_layers=2,
                              final_layer_builder=fb_1d_loss_builder,
                              gaussian_noise=0.2, dropout=0.15, ),
                         per_param_opts_fn=lr_strategy,
                         optimiser_class=optimizer, optimiser_params=optimizer_params)



In [12]:
data["metadata"]["use_images"] = True

n_tests = 1
multi_eval=False 
kfold=False
results, prfs = train_validate_ntimes(model_fn, data, batch_size, epochs, 
                                      augmentation_weights=augmentation_weights, 
                                      kfold=kfold, multi_eval=multi_eval,
                                      scheduler_init_fn=scheduler_init_fn, 
                                      validation_epochs=[2, 4, 7, 9, 11, 14, 17, 20, 23])
r1, p1 = results, prfs
results
prfs

# 21m 41s 7528192 0.807	0.700



Model Params = 16061504 
 MultiImageMultiTextAttentionEarlyFusionModel(
  (im_models): ModuleDict(
    (torchvision_resnet18): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stri


Epoch =  1 Loss = 0.895352 LR = 0.00000011



Epoch =  2 Loss = 0.850416 LR = 0.00000022
Epoch =  2 Train = 0.608819 Val = 0.579432



Epoch =  3 Loss = 0.822421 LR = 0.00000033



Epoch =  4 Loss = 0.801621 LR = 0.00000044
Epoch =  4 Train = 0.617081 Val = 0.596087


...


Epoch =  18 Loss = 0.628336 LR = 0.00000003



Epoch =  19 Loss = 0.614182 LR = 0.00000001



Epoch =  20 Loss = 0.617311 LR = 0.00000000
Epoch =  20 Train = 0.792747 Val = 0.712222



train   val
mean_or_std metric               
mean        map       0.685 0.583
            accuracy  0.720 0.652
            auc       0.793 0.712
std         map       0.000 0.000
            accuracy  0.000 0.000
            auc       0.000 0.000

train                             val                      
    precision recall    f1  supoort precision recall    f1  supoort
neg     0.830  0.702 0.761 4560.000     0.763  0.654 0.704 1140.000
pos     0.594  0.751 0.663 2640.000     0.520  0.648 0.577  660.000

In [12]:
epochs = 25
model_fn = model_builder(MultiImageMultiTextAttentionEarlyFusionModel, 
                         dict(image_models=[{"model": "torchvision_resnet50_swsl", "large_rf": True, "finetune": False}], num_classes=2, 
                              text_models=[dict(cls=Fasttext1DCNNModel, params=dict(classifier_dims=256, 
                                                                                    num_classes=2, n_tokens_out=16,
                                                                                    n_layers=2, final_layer_builder=lambda *args: None,
                                                                                    gaussian_noise=0.2, dropout=0.3, 
                                                                                    embedding_dims=256, internal_dims=256, featurizer="cnn",),
                                               in_channels=256, in_tokens=16)],
                              internal_dims=256, classifier_dims=256,
                              n_tokens_out=16, n_layers=2,
                              final_layer_builder=fb_1d_loss_builder,
                              gaussian_noise=0.3, dropout=0.4, ),
                         optimiser_class=optimizer, optimiser_params=optimizer_params)

data["metadata"]["use_images"] = True

n_tests = 1
multi_eval=False 
kfold=False
results, prfs = train_validate_ntimes(model_fn, data, batch_size, epochs, 
                                      augmentation_weights=augmentation_weights, 
                                      kfold=kfold, multi_eval=multi_eval,
                                      scheduler_init_fn=scheduler_init_fn, 
                                      validation_epochs=[2, 4, 7, 9, 11, 14, 17, 20, 23])
r1, p1 = results, prfs
results
prfs

# 21m 41s 7528192 0.807	0.700




Using cache found in /home/ahemf/.cache/torch/hub/facebookresearch_semi-supervised-ImageNet1K-models_master



Model Params = 7195136 
 MultiImageMultiTextAttentionEarlyFusionModel(
  (im_models): ModuleDict(
    (torchvision_resnet50_swsl): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, a

Exception for:  keyboard | '' '' | ['half_cut' 'word_cutout' 'text_rotate' 'keyboard'] argument of type 'NoneType' is not iterable

Epoch =  1 Loss = 1.257086 LR = 0.00006682



Epoch =  2 Loss = 1.124354 LR = 0.00013364
Epoch =  2 Train = 0.590686 Val = 0.574848


Exception for:  keyboard | ' ' ' ' ' ' | ['sentence_shuffle' 'word_cutout' 'half_cut' 'char_swap' 'keyboard'] argument of type 'NoneType' is not iterable

Epoch =  3 Loss = 1.058392 LR = 0.00020046


...


Epoch =  23 Loss = 0.640756 LR = 0.00001591
Epoch =  23 Train = 0.736269 Val = 0.613188



Epoch =  24 Loss = 0.649748 LR = 0.00000401



Epoch =  25 Loss = 0.652247 LR = 0.00000000



train   val
mean_or_std metric               
mean        map       0.615 0.483
            accuracy  0.643 0.558
            auc       0.738 0.615
std         map       0.000 0.000
            accuracy  0.000 0.000
            auc       0.000 0.000

train                             val                      
    precision recall    f1  supoort precision recall    f1  supoort
neg     0.812  0.567 0.668 4560.000     0.714  0.505 0.592 1140.000
pos     0.508  0.773 0.613 2640.000     0.432  0.650 0.519  660.000

# Predict

In [ ]:
batch_size=512
epochs = 1

submission, text_model = train_and_predict(model_fn, data, batch_size, epochs, augmentation_weights, scheduler_init_fn=scheduler_init_fn)
submission.to_csv("submission.csv",index=False)
submission.sample(3)


In [ ]:
submission.sample(10)